In [52]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import box, Point
import osmnx as ox

def download_and_plot_buildings(min_lon, min_lat, max_lon, max_lat, output_file, custom_points):
    # Create a bounding box
    bbox = box(min_lon, min_lat, max_lon, max_lat)

    # Download building footprints from OpenStreetMap
    buildings = ox.geometries_from_bbox(north=max_lat, south=min_lat, east=max_lon, west=min_lon, tags={'building': True})

     # Filter out non-polygon geometries
    buildings = buildings[buildings.geometry.type.isin(['Polygon', 'MultiPolygon'])]


    # Convert to GeoDataFrame
    gdf = gpd.GeoDataFrame(buildings, crs='EPSG:4326')

    # Download Georgia Tech campus boundary
    gt_campus = ox.geometries_from_place("Georgia Institute of Technology, Atlanta, Georgia", tags={'amenity': 'university'})
    gt_campus = gt_campus[gt_campus.geometry.type.isin(['Polygon', 'MultiPolygon'])]

    # Create a GeoDataFrame for custom points
    points_gdf = gpd.GeoDataFrame(
        custom_points, 
        geometry=[Point(x, y) for x, y in [(p['lon'], p['lat']) for p in custom_points]],
        crs='EPSG:4326'
    )

    # Plot the buildings and campus
    fig, ax = plt.subplots(figsize=(5,5))
    gdf.plot(ax=ax, edgecolor='grey', facecolor='snow', linewidth=0.5)
    gt_campus.plot(ax=ax, facecolor='none', edgecolor='blue', linewidth=0.75)
    
    # Plot only the custom points
    for idx, point in points_gdf.iterrows():
        ax.plot(point.geometry.x, point.geometry.y, 
                marker='o', 
                color=point['color'], 
                markersize=8, 
                label=point['name'])

    # Add a title
    plt.title('Georgia Tech Campus Map', fontsize=12)

    # Add latitude and longitude to axes
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')

    # Format the tick labels
    ax.xaxis.set_major_formatter(plt.FormatStrFormatter('%.3f'))
    ax.yaxis.set_major_formatter(plt.FormatStrFormatter('%.3f'))

    # Rotate x-axis labels for better readability
    plt.xticks(rotation=0)

    # Add legend
    plt.legend(loc="best", fontsize=8)

    # Adjust layout to prevent cutting off labels
    plt.tight_layout()

    # Save the plot as a PDF
    plt.savefig(output_file, format='pdf', dpi=300, bbox_inches='tight')
    plt.close()

    print(f"Map with building footprints, Georgia Tech campus, and custom points has been saved to {output_file}")

# Example usage:
min_lon, min_lat, max_lon, max_lat = -84.4075, 33.766, -84.3875, 33.782  # Coordinates for Georgia Tech area

# Define custom points
custom_points = [
    {'name': 'Reference Weather Station (CRC)', 'lat': 33.7756, 'lon': -84.3963, 'color': 'orange'},
    {'name': 'Science Square', 'lat': 33.7717, 'lon': -84.3949, 'color': 'purple'},
    {'name': 'Arts Square', 'lat': 33.7737, 'lon': -84.3985, 'color': 'teal'}
]

download_and_plot_buildings(min_lon, min_lat, max_lon, max_lat, 'georgia_tech_map_with_points.pdf', custom_points)

C:\Users\pkastner\AppData\Local\Temp\ipykernel_29936\1658795324.py:11: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  buildings = ox.geometries_from_bbox(north=max_lat, south=min_lat, east=max_lon, west=min_lon, tags={'building': True})
c:\Users\pkastner\anaconda3\envs\datascience\lib\site-packages\osmnx\geometries.py:48: FutureWarning: The `north`, `south`, `east`, and `west` parameters are deprecated and will be removed in the v2.0.0 release. Use the `bbox` parameter instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  return features.features_from_bbox(north, south, east, west, tags=tags)
c:\Users\pkastner\anaconda3\envs\datascience\lib\site-packages\o

Map with building footprints, Georgia Tech campus, and custom points has been saved to georgia_tech_map_with_points.pdf


In [50]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import box, Point
import osmnx as ox
import contextily as ctx

def download_and_plot_buildings(min_lon, min_lat, max_lon, max_lat, output_file, custom_points):
    # Create a bounding box
    bbox = box(min_lon, min_lat, max_lon, max_lat)

    # Download building footprints from OpenStreetMap
    buildings = ox.geometries_from_bbox(north=max_lat, south=min_lat, east=max_lon, west=min_lon, tags={'building': True})
    buildings = buildings[buildings.geometry.type.isin(['Polygon', 'MultiPolygon'])]

    # Convert to GeoDataFrame and reproject to Web Mercator
    gdf = gpd.GeoDataFrame(buildings, crs='EPSG:4326').to_crs(epsg=3857)

    # Download Georgia Tech campus boundary
    gt_campus = ox.geometries_from_place("Georgia Institute of Technology, Atlanta, Georgia", tags={'amenity': 'university'})
    gt_campus = gt_campus[gt_campus.geometry.type.isin(['Polygon', 'MultiPolygon'])].to_crs(epsg=3857)

    # Create a GeoDataFrame for custom points
    points_gdf = gpd.GeoDataFrame(
        custom_points, 
        geometry=[Point(x, y) for x, y in [(p['lon'], p['lat']) for p in custom_points]],
        crs='EPSG:4326'
    ).to_crs(epsg=3857)

    # Create the plot
    fig, ax = plt.subplots(figsize=(12, 12))

    # Plot the buildings and campus
    gdf.plot(ax=ax, edgecolor='grey', facecolor='lightblue', alpha=0.5, linewidth=0.5)
    gt_campus.plot(ax=ax, facecolor='none', edgecolor='blue', linewidth=2)
    
    # Plot custom points
    for idx, point in points_gdf.iterrows():
        ax.plot(point.geometry.x, point.geometry.y, 
                marker='o', 
                color=point['color'], 
                markersize=100, 
                label=point['name'])

    # Add the base map
    ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, zoom=15)

    # Add a title
    plt.title('Georgia Tech Campus Map', fontsize=16)

    # Remove axis labels as they're not needed with the base map
    ax.set_axis_off()

    # Add legend
    plt.legend(loc="upper left", fontsize=10, bbox_to_anchor=(0, 1), bbox_transform=ax.transAxes)

    # Adjust layout
    plt.tight_layout()

    # Save the plot as a PDF
    plt.savefig(output_file, format='pdf', dpi=300, bbox_inches='tight')
    plt.close()

    print(f"Map with building footprints, Georgia Tech campus, and custom points has been saved to {output_file}")

# Example usage:
min_lon, min_lat, max_lon, max_lat = -84.4075, 33.766, -84.3875, 33.782  # Coordinates for Georgia Tech area

# Define custom points
custom_points = [
    {'name': 'Reference Weather Station (CRC)', 'lat': 33.7756, 'lon': -84.3963, 'color': 'orange'},
    {'name': 'Science Square', 'lat': 33.7717, 'lon': -84.3949, 'color': 'purple'},
    {'name': 'Arts Square', 'lat': 33.7737, 'lon': -84.3985, 'color': 'teal'}
]

download_and_plot_buildings(min_lon, min_lat, max_lon, max_lat, 'georgia_tech_map.pdf', custom_points)

C:\Users\pkastner\AppData\Local\Temp\ipykernel_29936\28325138.py:12: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  buildings = ox.geometries_from_bbox(north=max_lat, south=min_lat, east=max_lon, west=min_lon, tags={'building': True})
c:\Users\pkastner\anaconda3\envs\datascience\lib\site-packages\osmnx\geometries.py:48: FutureWarning: The `north`, `south`, `east`, and `west` parameters are deprecated and will be removed in the v2.0.0 release. Use the `bbox` parameter instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  return features.features_from_bbox(north, south, east, west, tags=tags)
c:\Users\pkastner\anaconda3\envs\datascience\lib\site-packages\osm

Map with building footprints, Georgia Tech campus, and custom points has been saved to georgia_tech_map.pdf


In [160]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import box, Point
import osmnx as ox
import numpy as np

def create_tile_grid(min_lon, min_lat, max_lon, max_lat, num_tiles_x= 10, num_tiles_y=10):
    lon_edges = np.linspace(min_lon, max_lon, num_tiles_x + 1)
    lat_edges = np.linspace(min_lat, max_lat, num_tiles_y + 1)
    
    tiles = []
    for i in range(num_tiles_x):
        for j in range(num_tiles_y):
            tile = box(lon_edges[i], lat_edges[j], lon_edges[i+1], lat_edges[j+1])
            tiles.append(tile)
    
    return gpd.GeoDataFrame(geometry=tiles, crs='EPSG:4326')

def download_and_plot_buildings(min_lon, min_lat, max_lon, max_lat, output_file, custom_points):
    # Create a bounding box
    bbox = box(min_lon, min_lat, max_lon, max_lat)

    # Download building footprints from OpenStreetMap
    buildings = ox.geometries_from_bbox(north=max_lat, south=min_lat, east=max_lon, west=min_lon, tags={'building': True})

    # Filter out non-polygon geometries
    buildings = buildings[buildings.geometry.type.isin(['Polygon', 'MultiPolygon'])]

    # Convert to GeoDataFrame
    gdf = gpd.GeoDataFrame(buildings, crs='EPSG:4326')

    # Download Georgia Tech campus boundary
    gt_campus = ox.geometries_from_place("Georgia Institute of Technology, Atlanta, Georgia", tags={'amenity': 'university'})
    gt_campus = gt_campus[gt_campus.geometry.type.isin(['Polygon', 'MultiPolygon'])]

    # Create a GeoDataFrame for custom points
    points_gdf = gpd.GeoDataFrame(
        custom_points, 
        geometry=[Point(x, y) for x, y in [(p['lon'], p['lat']) for p in custom_points]],
        crs='EPSG:4326'
    )

    # Create tile grid
    offset_x =  0.0009
    offset_y =  0.00135
    num_tiles = 9
    tiles = create_tile_grid(min_lon+offset_x, min_lat+offset_y, max_lon+offset_x, max_lat+offset_y, num_tiles, num_tiles)

    # Plot the buildings and campus
    fig, ax = plt.subplots(figsize=(5,5))

    # Plot tiles
    tiles.plot(ax=ax, edgecolor='lightgrey', facecolor='whitesmoke', linewidth=0.5)

    gdf.plot(ax=ax, edgecolor='grey', facecolor='snow', linewidth=0.5)
    gt_campus.plot(ax=ax, facecolor='none', edgecolor='blue', linewidth=0.75)
    
    # Plot only the custom points
    for idx, point in points_gdf.iterrows():
        ax.plot(point.geometry.x, point.geometry.y, 
                marker='o', 
                color=point['color'], 
                markersize=8, 
                label=point['name'])

    # Add a title
    plt.title('Georgia Tech Campus Map', fontsize=12)

    # Add latitude and longitude to axes
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')

    # Format the tick labels
    ax.xaxis.set_major_formatter(plt.FormatStrFormatter('%.3f'))
    ax.yaxis.set_major_formatter(plt.FormatStrFormatter('%.3f'))

    # Rotate x-axis labels for better readability
    plt.xticks(rotation=0)

    # Add legend
    plt.legend(loc="best", fontsize=8)

    # Adjust layout to prevent cutting off labels
    plt.tight_layout()

    # Save the plot as a PDF
    plt.savefig(output_file+".pdf", format='pdf', bbox_inches='tight')
    plt.savefig(output_file+".png", format='png', dpi=1200, bbox_inches='tight')
    plt.close()

    print(f"Map with building footprints, Georgia Tech campus, and custom points has been saved to {output_file}")

# Example usage:
min_lon, min_lat, max_lon, max_lat = -84.4075, 33.766, -84.3875, 33.782  # Coordinates for Georgia Tech area

# Define custom points
custom_points = [
    {'name': 'Reference Weather Station (CRC)', 'lat': 33.7762, 'lon': -84.40353, 'color': 'orange'},
    {'name': 'Arts Square', 'lat': 33.7727, 'lon': -84.4007, 'color': 'teal'},
    {'name': 'Science Square', 'lat': 33.7689, 'lon': -84.4020, 'color': 'purple'}
]

download_and_plot_buildings(min_lon, min_lat, max_lon, max_lat, 'georgia_tech_map_with_tiles', custom_points)
download_and_plot_buildings(min_lon, min_lat, max_lon, max_lat, 'georgia_tech_map_with_tiles', custom_points)

C:\Users\pkastner\AppData\Local\Temp\ipykernel_29936\2113832316.py:24: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  buildings = ox.geometries_from_bbox(north=max_lat, south=min_lat, east=max_lon, west=min_lon, tags={'building': True})
c:\Users\pkastner\anaconda3\envs\datascience\lib\site-packages\osmnx\geometries.py:48: FutureWarning: The `north`, `south`, `east`, and `west` parameters are deprecated and will be removed in the v2.0.0 release. Use the `bbox` parameter instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  return features.features_from_bbox(north, south, east, west, tags=tags)
c:\Users\pkastner\anaconda3\envs\datascience\lib\site-packages\o

Map with building footprints, Georgia Tech campus, and custom points has been saved to georgia_tech_map_with_tiles


C:\Users\pkastner\AppData\Local\Temp\ipykernel_29936\2113832316.py:24: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  buildings = ox.geometries_from_bbox(north=max_lat, south=min_lat, east=max_lon, west=min_lon, tags={'building': True})
c:\Users\pkastner\anaconda3\envs\datascience\lib\site-packages\osmnx\geometries.py:48: FutureWarning: The `north`, `south`, `east`, and `west` parameters are deprecated and will be removed in the v2.0.0 release. Use the `bbox` parameter instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  return features.features_from_bbox(north, south, east, west, tags=tags)
c:\Users\pkastner\anaconda3\envs\datascience\lib\site-packages\o

Map with building footprints, Georgia Tech campus, and custom points has been saved to georgia_tech_map_with_tiles
